In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.0-bin-hadoop2.7/')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DF-Operations').getOrCreate()

In [20]:
#from pyspark.sql.types import StructField,StructType,FloatType,DateType,LongType
#data_schema = [StructField("Date",DateType(),True),StructField("Open",FloatType(),True),
               #StructField("High",FloatType(),True),StructField("Low",FloatType(),True),StructField("Close",FloatType(),True),
               #StructField("Volume",LongType(),True),StructField("Adj_Close",FloatType(),True)]

#final_data_schema = StructType(data_schema)

stock_DF = spark.read.csv('/home/ubuntu/appl_stock.csv',header=True,inferSchema=True)

In [21]:
stock_DF.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [22]:
stock_DF.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [30]:
stock_DF.filter( ((stock_DF['Close'] < 500) & (stock_DF['Open'] > 400)) & (stock_DF['high'] == stock_DF['close']) ).select(['Date','OPen','high','CLOSE']).show()

+-------------------+------------------+------------------+------------------+
|               Date|              OPen|              high|             CLOSE|
+-------------------+------------------+------------------+------------------+
|2011-10-13 00:00:00|404.98000299999995|408.42998900000003|408.42998900000003|
|2011-10-14 00:00:00|        416.829994|        421.999992|        421.999992|
+-------------------+------------------+------------------+------------------+



In [31]:
stock_DF.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [40]:
result = stock_DF.filter( ((stock_DF['Close'] < 500) & (stock_DF['Open'] > 400)) & (stock_DF['high'] == stock_DF['close']) ).select(['Date','OPen','high','CLOSE']).collect()

In [51]:
#result[0]['Date']
row = result[0]
row

Row(Date=datetime.datetime(2011, 10, 13, 0, 0), OPen=404.98000299999995, high=408.42998900000003, CLOSE=408.42998900000003)

In [57]:
row.asDict()

{'Date': datetime.datetime(2011, 10, 13, 0, 0),
 'OPen': 404.98000299999995,
 'high': 408.42998900000003,
 'CLOSE': 408.42998900000003}

In [60]:
sales_DF = spark.read.csv('/home/ubuntu/sales_info.csv',inferSchema=True,header=True)

In [61]:
sales_DF.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [62]:
sales_DF.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [69]:
sales_DF.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [68]:
sales_DF.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [83]:
sales_DF.groupby('Company').sum().alias('Total Sum').show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [79]:
sales_DF.agg({'Company':'max'}).show()

+------------+
|max(Company)|
+------------+
|        MSFT|
+------------+



In [84]:
from pyspark.sql.functions import avg,stddev,concat

In [91]:
sales_DF.select(concat('sales','Person','Company')).show()

+------------------------------+
|concat(sales, Person, Company)|
+------------------------------+
|                  200.0SamGOOG|
|              120.0CharlieGOOG|
|                340.0FrankGOOG|
|                 600.0TinaMSFT|
|                  124.0AmyMSFT|
|              243.0VanessaMSFT|
|                   870.0CarlFB|
|                  350.0SarahFB|
|                 250.0JohnAPPL|
|                130.0LindaAPPL|
|                 750.0MikeAPPL|
|               350.0 ChrisAPPL|
+------------------------------+



In [100]:
from pyspark.sql.functions import format_number
sales_DF.select(stddev('sales').alias('STANDARD_DEV')).select(format_number('STANDARD_DEV',2).alias('final')).show()
#result.select(format_number('STANDARD_DEV',3)).show()

+------+
| final|
+------+
|250.09|
+------+

